# NRL Comp setup

## Setting up markets and bots:
- On clean Heroku database
- Create a user for createMarket and for the bots.
- Set up markets on Monday night. Time is ISO8601 with a UTC offset to go to Brisbane time. Outcome between 0 and 10000 (10000 = StGeorge win, 0 = NewCastle win )
- Setup bot with BetfairPriceMakingBot(marketId, runnerId) looks at the market description and sets itself up. These will keep quoting until they encounter an error or run out of money. If they go down, put up again manually.
- Settle all markets at the end of the week.



In [1]:
# Auto-reload after edits
%load_ext autoreload
%autoreload 2

# Basic bot 
import requests
import pandas as pd
import json
import threading
from BlocBot import BlocBot
from BetfairPriceMakingBot import BetfairPriceMakingBot
import betfair as bf


In [2]:
# Create a new user
baseurl = 'http://127.0.0.1:5000/'
#baseurl = 'https://blocmarket.herokuapp.com/'
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content))

In [3]:
response.json()

{'signingKey': '242723508bfa23dc5af33d4d29a9852286dcdbbbf264a8c8f2ac3b379f57e493',
 'traderId': '23',
 'verifyKey': '7d0b3f8e81648619fd62380d50b2f3a6db49c8856d313d72c8a397158b16d67e'}

In [4]:
# Save user details
sk = response.json()['signingKey']
vk = response.json()['verifyKey']
tId = int(response.json()['traderId'])

In [5]:
# Set up event in spevent -> manual (Note: NEVER DO THIS. Get the bot to pick up the market details.)
baseurl = 'http://127.0.0.1:5000/'


url = baseurl + 'createSPEvent'
headers = {'content-type': 'application/json'}

content_makeevent = {"sport": "Rugby League",
                        "competition": "NRL",
                        "runners": json.dumps({"0": "Newcastle", "1": "NZ warriors"}), 
                        "starttimestamputc": "6-Jul-2019"}
# Post market
response = requests.post(url, data=json.dumps(content_makeevent), headers=headers)

In [6]:
eventid = response.json()['eventid']
eventid

'5'

In [23]:
# Set up market in spmarket (for money line -> set betfairid in marketparameters json )
url = baseurl + 'createSPMarket'

content_makemarket = {"eventid": eventid,
                        "markettype": "MATCH_ODDS", 
                        "marketparameters": json.dumps({'source': {'betfair': {'marketid':'1.159418817'}}}),
                        "notes": ""}
# Post market
response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)


In [24]:
spmarketid = response.json()['marketid']
spmarketid

'8'

In [25]:
# Set up market with marketDesc as 'spmarketid: 8' etc (Use this for bots to pick up)
url = baseurl + 'createMarket'
headers = {'content-type': 'application/json'}

content_makemarket = {"signingKey": sk,
                        "traderId": tId, 
                        "verifyKey": vk,
                        "marketRootId": 14, 
                        "marketBranchId": 1, 
                        "marketMin": 0,
                        "marketMax": 10000,
                        "marketDesc": json.dumps({'spmarket': {'marketid': spmarketid , 'runnerid': 0}})}
# Post market
response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)


In [26]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '6', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 1,
 'marketId': 6,
 'marketMax': 10000,
 'marketMin': 0,
 'marketRootId': 14,
 'traderId': 23}

In [27]:
marketId = response.json()['marketId']
marketId

6

In [38]:
bot = BetfairPriceMakingBot(marketId=6, runnerId = 0, blocUrl = 'http://127.0.0.1:5000/')

In [39]:
vars(bot)

{'multiplier': 10000,
 'spread': 0.01,
 'updateFrequencySeconds': 180,
 'verifyKey': [],
 'signingKey': [],
 'traderId': [],
 'marketId': 6,
 'blocurl': 'http://127.0.0.1:5000/',
 'blocheaders': {'content-type': 'application/json'},
 'quoteSource': 'betfair',
 'betfairMarketId': '1.159049810',
 'betfairRunnerId': 0,
 'betfairAppKey': '',
 'betfairSessionKey': '',
 'spmarketid': 7,
 'blocUrl': 'http://127.0.0.1:5000/'}

In [40]:
# Basic Bot definition
botdef = {'spread': 0.01,
 'updateFrequencySeconds': 30,
 'verifyKey': vk,
 'signingKey':  sk,
 'traderId': tId,
 'multiplier': 10000,
 'blocurl': baseurl,
 'blocheaders': {'content-type': 'application/json'}}

bot.getBetfairSessionToken(betfairAppKey ='iw8UsiHCP1GSs213', betfairPassword='ee')
# Add some defs
for key,val in botdef.items():
    # Set bot inputs from definiton
    setattr(bot, key, val)

In [ ]:
vars(bot)

In [41]:
bot.betfairMarketId = '1.159418817'

In [50]:
quote, bids, asks = bot.getQuote()
bestAsk = asks[asks.price==min(asks.price)]
bestBid = bids[bids.price==max(bids.price)]

In [70]:
# Get bot to save odds/stake as sp records AND make prices in local bloc market as probability 
# Write up this process and prep for live multi-market bots.

In [51]:
bids

,price,size
0,1.38,581.05
1,1.37,483.50
2,1.36,1154.94


In [52]:
asks

,price,size
0,1.40,3684.16
1,1.41,3126.46
2,1.42,2250.16


In [47]:
bestBid

,price,size
0,1.38,531.05


In [48]:
bestAsk

,price,size
0,1.4,3815.56


In [53]:
bot.scrapeQuote(bestBid, bestAsk)

In [54]:
bot.updateFrequencySeconds = 5

In [ ]:
bot.streamQuote(scrapeOdds=True, postQuotes=True)

[Within cells interlinked]: quote 7194.616977225674
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: quote 7220.310707955376
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: No price change.
[Within cells interlinked]: quote 7194.616977225674
[Within cells interlinked]: No price change.
[Within cells interlinked]: quote 7220.310707955376
[Within cells interlinked]: No price change.
[Within cells interlinked]:

In [33]:
# Create all the markets

url = baseurl + 'createMarket'
headers = {'content-type': 'application/json'}

# Market descriptions 
marketD = ['Event Melbourne v Newcastle 2019-06-15T16:00:00-10:00 betfair 1.159515068',
          'Event SouthSydney v Penrith 2019-06-15T16:00:00-10:00 betfair 1.159512658']
          
mrtId = 0
allMarketIds = []
for md in marketD:
    mrtId = mrtId+1
    content_makemarket = {"signingKey": sk,
                        "traderId": tId, 
                        "verifyKey": vk,
                        "marketRootId": mrtId, 
                        "marketBranchId": 1, 
                        "marketMin": 0,
                        "marketMax": 10000,
                        "marketDesc": md}
    # Post market
    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    allMarketIds.append(response.json()['marketId'])


In [34]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '5', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 1,
 'marketId': 5,
 'marketMax': 10000,
 'marketMin': 0,
 'marketRootId': 6,
 'traderId': 12}

In [ ]:
# Basic Bot definition
botdef = {'spread': 0.01,
 'updateFrequencySeconds': 30,
 'verifyKey': vk,
 'signingKey':  sk,
 'traderId': tId,
 'multiplier': 10000,
 'blocurl': baseurl,
 'blocheaders': {'content-type': 'application/json'}}

botdef

In [78]:
str(datetime.datetime.utcnow())

'2019-06-23 07:54:14.721062'

# Run single bot

In [ ]:
# Create a bot for market (Make sure VPN location is OK)
iMarket = 1
bot = BetfairPriceMakingBot(iMarket)
bot.getBetfairSessionToken(betfairAppKey ='iw8UsiHCP1GSs213', betfairPassword=bfPass)
    
# Add some defs
for key,val in botdef.items():
    # Set bot inputs from definiton
    setattr(bot, key, val)
    


In [ ]:
# Check quot
bot.getQuote()

In [39]:
# Stream qutoe
bot.streamQuotes()

TypeError: scrapeQuote() missing 2 required positional arguments: 'bid' and 'ask'

In [62]:
import datetime
datetime.datetime.utcnow()

datetime.datetime(2019, 6, 23, 7, 38, 45, 296024)

# Run multiple bots on threads


In [ ]:
# Set up bot threads
threads = []
for iMarket in range(len(marketD)):
    
    # Create a bot for each market (Make sure VPN location is OK)
    bot = BetfairPriceMakingBot(iMarket+1)
    bot.getBetfairSessionToken(betfairAppKey ='iw8UsiHCP1GSs213', betfairPassword=bfPass)
    
    # Add some defs
    for key,val in botdef.items():
        # Set bot inputs from definiton
        setattr(bot, key, val)
    
    t = threading.Thread(target=bot.streamQuote, args=())
    threads.append(t)


In [ ]:
# Start threads
for t in threads:
    t.start()
    

In [ ]:
threads

In [ ]:
bot

In [ ]:
vars(bot)

In [ ]:
pd.read_json(bot.getTradeSummary())

In [ ]:
bot.getQuote()

In [ ]:
vars(bot)

In [28]:
response

<Response [500]>